Rebuild street graph
====================
Loads a street graph and reallocates roads space, e.g., into a network of one-way streets.

Prerequisite: Preparing the simplified street graph

In [ ]:
import snman, os
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

#CRS_internal = 29119    # for Boston
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

In [ ]:
print('Load perimeters')
perimeters_gdf = snman.io.load_perimeters(os.path.join(inputs_path, 'perimeters', 'perimeters.shp'), crs=CRS_internal)

print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    only_active=False,
    filter_ids=['zrh-main', 'zrh-4', 'zrh-6'],
    crs=CRS_internal
)

In [ ]:
snman.rebuilding.multi_rebuild_regions(
    G, 
    rebuilding_regions_gdf,
    width_attribute=KEY_LANES_DESCRIPTION + '_width_total_m',
    rebuilding_function=snman.rebuilding.multi_rebuild,
    given_lanes_function=snman.rebuilding.multi_set_given_lanes,
    public_transit_mode='mandatory_like_existing',
    parking_mode='mandatory_like_existing',
    needed_node_access_function=snman.rebuilding.multi_set_needed_node_access,
    existing_lanes_attribute=KEY_LANES_DESCRIPTION,
    given_lanes_attribute=KEY_GIVEN_LANES_DESCRIPTION,
    target_lanes_attribute=KEY_LANES_DESCRIPTION_AFTER,
    export_L=(export_path + 'L_edges.gpkg', export_path + 'L_nodes.gpkg'),
    export_H=(export_path + 'H_edges.gpkg', export_path + 'H_nodes.gpkg'),
    export_when='after',
    verbose=True
)

In [ ]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

In [ ]:
if 1:
    snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
    snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

Add edge costs
--------------
We add the mode-specific cost to the edges so that they can be easily inspected

In [ ]:
print('Add edge costs')
snman.street_graph.add_edge_costs(G, lanes_description=KEY_LANES_DESCRIPTION)
snman.street_graph.add_edge_costs(G, lanes_description=KEY_LANES_DESCRIPTION_AFTER)

Export
------
Saves the resulting datasets to the disk. Use the *snman_detailed.qgz* file to view them in QGIS.

In [ ]:
if 1:
    print('Export network without lanes')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'street_graph_edges.gpkg'),
        os.path.join(export_path, 'street_graph_nodes.gpkg'),
        crs=CRS_for_export
    )

if 1:
    print('Export lane geometries')
    SCALING = 3
    snman.io.export_street_graph_with_lanes(
        G,
        [KEY_LANES_DESCRIPTION, KEY_LANES_DESCRIPTION_AFTER],
        os.path.join(export_path, 'lane_geometries.shp'),
        scaling=SCALING,
        crs=CRS_for_export
    )

if 1:
    print('Export lane graphs')
    L_before = snman.lane_graph.create_lane_graph(G, KEY_LANES_DESCRIPTION)
    L_after = snman.lane_graph.create_lane_graph(G, KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_street_graph(
        L_before,
        os.path.join(export_path, 'lane_graph_edges.gpkg'),
        os.path.join(export_path, 'lane_graph_nodes.gpkg'),
        crs=CRS_for_export
    )
    snman.io.export_street_graph(
        L_after,
        os.path.join(export_path, 'lane_graph_after_edges.gpkg'),
        os.path.join(export_path, 'lane_graph_after_nodes.gpkg'),
        crs=CRS_for_export
    )
    